In [5]:
import numpy as np

import astropy.io.fits as fits

import glob

# Holy shit it worked.

from lsst.sims.photUtils.Sed import Sed

In [24]:
dir = '/Users/ichorgodbel-shamroth/lsst_stack/fits_files/ck04models/ckm05/'
filepath = glob.glob(dir+'*.fits')

In [38]:
for filename in filepath:
    spectra_data = fits.getdata(filename)

In [37]:
# Progress!
for filename in filepath:
    spectrum = fits.open(filename)